In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import mediapipe as mp

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import feedforward

## Setup folder and register the training data

In [2]:
VIDEOS_PATH = os.path.join('dist/videos') 
NUM_VIDEOS = 30
NUM_FRAMES_PER_VIDEO = 100
feedforward.create_folder_if_not_exists(VIDEOS_PATH)
feedforward.clear_folder(VIDEOS_PATH)
video_prefix = 'arrow'
feedforward.register_videos(VIDEOS_PATH, video_prefix, NUM_VIDEOS, NUM_FRAMES_PER_VIDEO, 'beautiful_gesture')

2023-12-09 13:38:58.311 Python[46867:5999257] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 

## Classification

### Data Augmentation

In [ ]:
augmented_prefix = "augmented_rotation_"
feedforward.augment_dataset_rotation(VIDEOS_PATH, augmented_prefix)

In [ ]:
#feedforward.delete_files_with_prefix(VIDEOS_PATH, augmented_prefix)

### Building the training set

In [ ]:
WINDOW_LEN = 20
TARGET_LANDMARKS = [
    mp.solutions.holistic.PoseLandmark.LEFT_SHOULDER,
    mp.solutions.holistic.PoseLandmark.LEFT_HIP,
]
X_training, y_training = feedforward.build_training_arrow_data(VIDEOS_PATH, WINDOW_LEN, TARGET_LANDMARKS)
X_training.shape, y_training.shape

### Build and Train the model

In [ ]:
X_training.shape[1], X_training.shape[2]


In [ ]:
model = Sequential()

NUM_OUTPUT_UNITS = len(TARGET_LANDMARKS) * 2

model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True)) # LSTM layer with input shape (X_training.shape[1], X_training.shape[2]))
model.add(LSTM(64, activation='relu', return_sequences=False)) # LSTM layer with return_sequences=False for the final prediction# LSTM layer with return_sequences=False for the final prediction
model.add(Dense(NUM_OUTPUT_UNITS))# Dense output layer with NUM_OUTPUT_UNITS units

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

history = model.fit(X_training, y_training, batch_size=32, epochs=100, validation_split=0.2)

model.summary()

## Test

In [ ]:
cap = cv2.VideoCapture(feedforward.CAMERA_INDEX)
holistic = mp.solutions.holistic.Holistic()
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = holistic.process(frame_rgb)

    if results.pose_landmarks:

        pixel_coords = []
        height, width, _ = frame.shape
        data = []

        for target_landmark in TARGET_LANDMARKS:
            target_coords = results.pose_landmarks.landmark[target_landmark]
            target_input = [target_coords.x,  target_coords.y]
            data += target_input

            pixel_coords_x, pixel_coords_y = int(target_coords.x * width), int(target_coords.y * height)
            pixel_coords.append([pixel_coords_x, pixel_coords_y])
            cv2.circle(frame, (pixel_coords_x, pixel_coords_y), 5, (255, 0, 0), -1)

        nose_coords = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
        nose_input = [nose_coords.x, nose_coords.y]
        data += nose_input
        frame_buffer.append(data)

        if len(frame_buffer) >= WINDOW_LEN/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)
    
            prediction = np.array(np.split(prediction[0], len(TARGET_LANDMARKS)))
            print(prediction)

            for i, target_landmark in enumerate(TARGET_LANDMARKS):
                print(i)
                prediction_x = int(prediction[i][0] * width)
                prediction_y = int(prediction[i][1] * height)
            
                cv2.arrowedLine(frame, (pixel_coords[i][0], pixel_coords[i][1]), (prediction_x, prediction_y), (255, 0, 255), 2)

            frame_buffer = frame_buffer[1:]

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()